In [ ]:
import sqlite3
import numpy as np
import pandas as pd
from sqlite3 import Error
import math
import blpapi
import pdblp
from datetime import date

DB = PySQLite("test1228v1.db")

class CorporateCredit():    
    def __init__(self,name):     
        self.name = name
    
    def FCF(self,qtrs):
        FCF = ['Revenue','Variable Costs','Fixed Costs','EBITDA', 'Less: Capex', 'Less: Working Cap', 'Less: Taxes', 'FCF', 'Beginning Cash', 'Ending Cash']

        revArray = np.array([self.CurrentFCF['rev']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
        vcostsArray = np.array([self.CurrentFCF['var_costs']*(1+self.CurrentFCF['rev_g'])**i for i in range(qtrs+1)],dtype=int)
        fcostsArray = np.full(shape=qtrs+1,fill_value=self.CurrentFCF['f_costs'],dtype=int)
        ebitdaArray = revArray+vcostsArray+fcostsArray
        capexArray = revArray * self.CurrentFCF['capex/rev']
        wcArray = revArray * self.CurrentFCF['WC/rev']
        taxArray = ebitdaArray * self.CurrentFCF['tax']  #to fix --> need interest exp
        fcfArray = ebitdaArray + capexArray + wcArray + taxArray # need interest expense

        begCashArray = np.zeros(qtrs+1,dtype=int)
        begCashArray[0] = self.CurrentFCF['cash']
        endCashArray = np.zeros(qtrs+1,dtype=int)
        endCashArray[0] = fcfArray[0]+begCashArray[0]


        for i in range(1,qtrs+1):
          begCashArray[i] = endCashArray[i-1]
          endCashArray[i] = begCashArray[i] + fcfArray[i]


        FCF_statement=[revArray,vcostsArray,fcostsArray,ebitdaArray,capexArray,wcArray,taxArray,fcfArray,begCashArray,endCashArray]
        FCF_df=pd.DataFrame(FCF_statement,columns = self.makeDateColumn(qtrs), index = FCF, dtype=int)

          # print(ebitdaArray)
          # print(begCashArray)
          # print(endCashArray)


        return FCF_df
        

def main():
    ETP=CorporateCredit('ETP')
    

    
main()
     